Простой пример: Светофор
Моделируем работу светофора, который переключается между состояниями: Красный, Желтый, Зеленый. Поведение (что делать дальше) зависит от текущего цвета.

In [1]:
from __future__ import annotations # Для type hints внутри класса
from abc import ABC, abstractmethod
import time

# --- Context (Контекст) ---
class TrafficLight:
    """Контекст: сам светофор."""
    _state: State | None = None # Ссылка на текущее состояние

    def __init__(self, initial_state: State):
        self.transition_to(initial_state) # Устанавливаем начальное состояние
        print(f"Traffic Light initialized in state: {type(initial_state).__name__}")

    def transition_to(self, state: State):
        """Метод для смены состояния."""
        print(f"Traffic Light: Transitioning to {type(state).__name__}...")
        self._state = state
        self._state.context = self # Передаем ссылку на контекст в новое состояние

    def request_change(self):
        """Запрос на смену сигнала (делегируется состоянию)."""
        print("Traffic Light: Requesting change...")
        if self._state:
            self._state.handle_request()
        else:
            print("Traffic Light: Error - No state set.")

    def get_current_color(self) -> str:
         """Возвращает текущий цвет (запрашивается у состояния)."""
         if self._state:
             return self._state.get_color()
         return "Unknown"

# --- State Interface (Интерфейс Состояния) ---
class State(ABC):
    """Абстрактное состояние."""
    _context: TrafficLight | None = None

    @property
    def context(self) -> TrafficLight | None:
        return self._context

    @context.setter
    def context(self, context: TrafficLight | None) -> None:
        self._context = context

    @abstractmethod
    def handle_request(self) -> None:
        """Обрабатывает запрос на смену состояния."""
        pass

    @abstractmethod
    def get_color(self) -> str:
        """Возвращает цвет, соответствующий состоянию."""
        pass

# --- Concrete States (Конкретные Состояния) ---
class RedState(State):
    """Состояние: Красный свет."""
    def handle_request(self) -> None:
        print("RedState: Handling request...")
        print("  Turning Green.")
        time.sleep(1) # Имитация задержки
        self.context.transition_to(GreenState()) # Переход в Зеленое состояние

    def get_color(self) -> str:
        return "Red"

class YellowState(State):
    """Состояние: Желтый свет."""
    def handle_request(self) -> None:
        print("YellowState: Handling request...")
        print("  Turning Red.")
        time.sleep(0.5) # Желтый горит недолго
        self.context.transition_to(RedState()) # Переход в Красное состояние

    def get_color(self) -> str:
        return "Yellow"

class GreenState(State):
    """Состояние: Зеленый свет."""
    def handle_request(self) -> None:
        print("GreenState: Handling request...")
        print("  Turning Yellow.")
        time.sleep(2) # Зеленый горит дольше
        self.context.transition_to(YellowState()) # Переход в Желтое состояние

    def get_color(self) -> str:
        return "Green"


# --- Client Code ---
if __name__ == "__main__":
    # Создаем светофор в начальном состоянии (Красный)
    # Передаем экземпляр, а не класс, т.к. состояния могут иметь свои данные
    traffic_light = TrafficLight(RedState())
    print(f"Initial color: {traffic_light.get_current_color()}")

    # Имитируем цикл работы светофора
    for _ in range(6): # Пройдем пару полных циклов
        print("-" * 20)
        traffic_light.request_change()
        print(f"Current color: {traffic_light.get_current_color()}")

# Вывод:
# Traffic Light: Transitioning to RedState...
# Traffic Light initialized in state: RedState
# Initial color: Red
# --------------------
# Traffic Light: Requesting change...
# RedState: Handling request...
#   Turning Green.
# Traffic Light: Transitioning to GreenState...
# Current color: Green
# --------------------
# Traffic Light: Requesting change...
# GreenState: Handling request...
#   Turning Yellow.
# Traffic Light: Transitioning to YellowState...
# Current color: Yellow
# --------------------
# Traffic Light: Requesting change...
# YellowState: Handling request...
#   Turning Red.
# Traffic Light: Transitioning to RedState...
# Current color: Red
# --------------------
# Traffic Light: Requesting change...
# RedState: Handling request...
#   Turning Green.
# Traffic Light: Transitioning to GreenState...
# Current color: Green
# --------------------
# Traffic Light: Requesting change...
# GreenState: Handling request...
#   Turning Yellow.
# Traffic Light: Transitioning to YellowState...
# Current color: Yellow
# --------------------
# Traffic Light: Requesting change...
# YellowState: Handling request...
#   Turning Red.
# Traffic Light: Transitioning to RedState...
# Current color: Red

Traffic Light: Transitioning to RedState...
Traffic Light initialized in state: RedState
Initial color: Red
--------------------
Traffic Light: Requesting change...
RedState: Handling request...
  Turning Green.
Traffic Light: Transitioning to GreenState...
Current color: Green
--------------------
Traffic Light: Requesting change...
GreenState: Handling request...
  Turning Yellow.
Traffic Light: Transitioning to YellowState...
Current color: Yellow
--------------------
Traffic Light: Requesting change...
YellowState: Handling request...
  Turning Red.
Traffic Light: Transitioning to RedState...
Current color: Red
--------------------
Traffic Light: Requesting change...
RedState: Handling request...
  Turning Green.
Traffic Light: Transitioning to GreenState...
Current color: Green
--------------------
Traffic Light: Requesting change...
GreenState: Handling request...
  Turning Yellow.
Traffic Light: Transitioning to YellowState...
Current color: Yellow
--------------------
Traffic L

Сложный пример: Статус Документа в Системе Документооборота
Моделируем документ, который проходит через разные стадии: Черновик -> На Модерации -> Опубликован -> Архивирован. Действия (публикация, отклонение, архивация) возможны только в определённых состояниях.

In [2]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Optional

# --- State Interface ---
class DocumentState(ABC):
    """Абстрактное состояние документа."""
    _document: 'Document' | None = None

    @property
    def document(self) -> 'Document':
        if self._document is None:
            raise ValueError("Document context not set for state")
        return self._document

    @document.setter
    def document(self, document: 'Document') -> None:
        self._document = document

    @abstractmethod
    def publish(self) -> None: pass
    @abstractmethod
    def archive(self) -> None: pass
    @abstractmethod
    def reject(self) -> None: pass # Добавим отклонение модерации
    @abstractmethod
    def get_status_name(self) -> str: pass


# --- Context (Контекст) ---
class Document:
    """Контекст: документ с изменяемым состоянием."""
    _state: DocumentState

    def __init__(self, content: str):
        self.content = content
        self.admin_permission = False # Флаг для усложнения логики
        # Начинаем в состоянии Черновика
        self.transition_to(DraftState())
        print(f"Document created. Initial state: {self._state.get_status_name()}")

    def transition_to(self, state: DocumentState):
        print(f"Document: Transitioning from {type(self._state).__name__ if hasattr(self, '_state') else 'None'} to {type(state).__name__}...")
        self._state = state
        self._state.document = self # Даем состоянию ссылку на себя

    # Методы документа, делегирующие выполнение состоянию
    def publish_document(self):
        print("Document: Attempting to publish...")
        self._state.publish()

    def archive_document(self):
        print("Document: Attempting to archive...")
        self._state.archive()

    def reject_document(self):
         print("Document: Attempting to reject...")
         self._state.reject()

    def get_status(self) -> str:
        return self._state.get_status_name()

# --- Concrete States ---
class DraftState(DocumentState):
    """Состояние: Черновик."""
    def publish(self) -> None:
        print("DraftState: Publishing...")
        print("  Moving document to Moderation.")
        self.document.transition_to(ModerationState()) # Переход на модерацию

    def archive(self) -> None:
        print("DraftState: Cannot archive a draft directly.")
        # Можно добавить переход в состояние ArchivedDraftState, если нужно

    def reject(self) -> None:
         print("DraftState: Cannot reject a draft.")

    def get_status_name(self) -> str: return "Draft"

class ModerationState(DocumentState):
    """Состояние: На модерации."""
    def publish(self) -> None:
        # Публикация требует прав администратора (усложнение)
        if self.document.admin_permission:
            print("ModerationState: Publishing (Admin override)...")
            print("  Document is now Published.")
            self.document.transition_to(PublishedState())
        else:
            print("ModerationState: Publish failed. Requires admin permission from moderation state.")

    def archive(self) -> None:
        print("ModerationState: Cannot archive while in moderation.")

    def reject(self) -> None:
         print("ModerationState: Rejecting moderation...")
         print("  Returning document to Draft state.")
         self.document.transition_to(DraftState()) # Возвращаем в черновики

    def get_status_name(self) -> str: return "Moderation"

class PublishedState(DocumentState):
    """Состояние: Опубликован."""
    def publish(self) -> None:
        print("PublishedState: Document is already published.")

    def archive(self) -> None:
        print("PublishedState: Archiving document...")
        self.document.transition_to(ArchivedState()) # Переход в архив

    def reject(self) -> None:
         print("PublishedState: Cannot reject an already published document.")

    def get_status_name(self) -> str: return "Published"

class ArchivedState(DocumentState):
    """Состояние: Архивирован."""
    def publish(self) -> None:
        print("ArchivedState: Cannot publish from archive. Unarchive first (not implemented).")

    def archive(self) -> None:
        print("ArchivedState: Document is already archived.")

    def reject(self) -> None:
         print("ArchivedState: Cannot reject an archived document.")

    def get_status_name(self) -> str: return "Archived"


# --- Client Code ---
if __name__ == "__main__":
    doc = Document("My important document content.")
    print(f"Status: {doc.get_status()}")

    print("\n--- Trying to archive draft ---")
    doc.archive_document() # Не сработает
    print(f"Status: {doc.get_status()}")

    print("\n--- Publishing draft ---")
    doc.publish_document() # Перейдет в Moderation
    print(f"Status: {doc.get_status()}")

    print("\n--- Trying to publish from moderation (no admin) ---")
    doc.publish_document() # Не сработает
    print(f"Status: {doc.get_status()}")

    print("\n--- Rejecting moderation ---")
    doc.reject_document() # Вернется в Draft
    print(f"Status: {doc.get_status()}")

    print("\n--- Publishing draft again ---")
    doc.publish_document() # Снова в Moderation
    print(f"Status: {doc.get_status()}")

    print("\n--- Granting admin permission and publishing ---")
    doc.admin_permission = True # Даем права
    doc.publish_document() # Теперь должно сработать -> Published
    print(f"Status: {doc.get_status()}")

    print("\n--- Trying to publish again ---")
    doc.publish_document() # Уже опубликован
    print(f"Status: {doc.get_status()}")

    print("\n--- Archiving document ---")
    doc.archive_document() # Перейдет в Archived
    print(f"Status: {doc.get_status()}")

    print("\n--- Trying to archive again ---")
    doc.archive_document() # Уже в архиве
    print(f"Status: {doc.get_status()}")

# Вывод:
# Document: Transitioning from None to DraftState...
# Document created. Initial state: Draft
# Status: Draft
#
# --- Trying to archive draft ---
# Document: Attempting to archive...
# DraftState: Cannot archive a draft directly.
# Status: Draft
#
# --- Publishing draft ---
# Document: Attempting to publish...
# DraftState: Publishing...
#   Moving document to Moderation.
# Document: Transitioning from DraftState to ModerationState...
# Status: Moderation
#
# --- Trying to publish from moderation (no admin) ---
# Document: Attempting to publish...
# ModerationState: Publish failed. Requires admin permission from moderation state.
# Status: Moderation
#
# --- Rejecting moderation ---
# Document: Attempting to reject...
# ModerationState: Rejecting moderation...
#   Returning document to Draft state.
# Document: Transitioning from ModerationState to DraftState...
# Status: Draft
#
# --- Publishing draft again ---
# Document: Attempting to publish...
# DraftState: Publishing...
#   Moving document to Moderation.
# Document: Transitioning from DraftState to ModerationState...
# Status: Moderation
#
# --- Granting admin permission and publishing ---
# Document: Attempting to publish...
# ModerationState: Publishing (Admin override)...
#   Document is now Published.
# Document: Transitioning from ModerationState to PublishedState...
# Status: Published
#
# --- Trying to publish again ---
# Document: Attempting to publish...
# PublishedState: Document is already published.
# Status: Published
#
# --- Archiving document ---
# Document: Attempting to archive...
# PublishedState: Archiving document...
# Document: Transitioning from PublishedState to ArchivedState...
# Status: Archived
#
# --- Trying to archive again ---
# Document: Attempting to archive...
# ArchivedState: Document is already archived.
# Status: Archived

Document: Transitioning from None to DraftState...
Document created. Initial state: Draft
Status: Draft

--- Trying to archive draft ---
Document: Attempting to archive...
DraftState: Cannot archive a draft directly.
Status: Draft

--- Publishing draft ---
Document: Attempting to publish...
DraftState: Publishing...
  Moving document to Moderation.
Document: Transitioning from DraftState to ModerationState...
Status: Moderation

--- Trying to publish from moderation (no admin) ---
Document: Attempting to publish...
ModerationState: Publish failed. Requires admin permission from moderation state.
Status: Moderation

--- Rejecting moderation ---
Document: Attempting to reject...
ModerationState: Rejecting moderation...
  Returning document to Draft state.
Document: Transitioning from ModerationState to DraftState...
Status: Draft

--- Publishing draft again ---
Document: Attempting to publish...
DraftState: Publishing...
  Moving document to Moderation.
Document: Transitioning from Draft